### Load Test Data and Trained APG Sampler

In [1]:
%matplotlib inline
import torch
import numpy as np
from apgs.gmm.apg_training import init_apg_models, init_rws_models

CUDA = torch.cuda.is_available()
device = torch.device('cuda:1')
data_dir = '../../../data/gmm/'
K, D, num_hidden = 3, 2, 32

data = torch.from_numpy(np.load(data_dir + 'ob_test.npy')).float() 
model_version = 'apg-gmm-block=decomposed-num_sweeps=5-num_samples=20'
models = init_apg_models(K, D, num_hidden, CUDA, device, model_version, lr=None)

### Visualize Samples

In [ ]:
from apgs.resampler import Resampler
from apgs.gmm.objectives import apg_objective
from apgs.gmm.evaluation import viz_samples

batch_size, num_sweeps = 5, 13
batch_data = data[torch.randperm(data.shape[0])[:batch_size]].unsqueeze(0)
if CUDA:
    batch_data = batch_data.cuda().to(device)
result_flags = {'loss_required' : False, 'ess_required' : False, 'mode_required' : True, 'density_required': False}
trace = apg_objective(models, batch_data, result_flags, num_sweeps, 'decomposed', Resampler('systematic', 1, CUDA, device))
viz_samples(batch_data.cpu().squeeze(), trace, num_sweeps, K, save_name=None)

### Computational Budget Analysis

In [ ]:
from apgs.gmm.evaluation import budget_analysis, plot_budget_analyais_results
budget = 1000
num_sweeps = np.array([1, 5, 10 , 20, 25])
sample_sizes = 1000 / num_sweeps
blocks = ['decomposed', 'joint']
df = budget_analysis(models, blocks, num_sweeps, sample_sizes, data, K, CUDA, device)
plot_budget_analyais_results(df)

### Comparison with Baselines

In [ ]:
from apgs.gmm.evaluation import density_convergence, plot_convergence
seed = 1
sample_size, num_sweeps, num_runs = 100, 30, 10
lp_step_size, lp_num_steps, bpg_factor = 1e-1, [1, 5, 10], 100
x = data[torch.randperm(data.shape[0])[0]].repeat(sample_size, 1, 1, 1)
if CUDA:
    x = x.cuda().to(device)
densities = density_convergence(models, x, K, num_runs, num_sweeps, lp_step_size, lp_num_steps, bpg_factor, CUDA, device)
plot_convergence(densities)